# This is the in situ and SSS collocation code. 


In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
import xarray as xr
import scipy
from glob import glob
import cartopy.crs as ccrs
from pyresample.geometry import AreaDefinition
from pyresample import image, geometry, load_area, save_quicklook, SwathDefinition, area_def2basemap
from pyresample.kd_tree import resample_nearest
from math import radians, cos, sin, asin, sqrt
from scipy import spatial
import os.path
from os import path
import gzip
import shutil


C:\Users\gentemann\Miniconda3\envs\satenv\lib\site-packages\xarray\core\merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


# Define a function to read in insitu data
- Read in the Saildrone USV file either from a local disc or using OpenDAP.
- add room to write collocated data to in situ dataset


In [2]:
def read_usv(iusv):
#    filename_usv_list = ['F:/data/cruise_data/saildrone/noaa_arctic/PMEL_2015/126/pmel_2015_sd126-ALL-1_min-v1.nc',
#                         'F:/data/cruise_data/saildrone/noaa_arctic/PMEL_2015/128/pmel_2015_sd128-ALL-1_min-v1.nc',
#                         'F:/data/cruise_data/saildrone/noaa_arctic/PMEL_2016/126/pmel_2016_sd126-ALL-1_min-v1.nc',
#                         'F:/data/cruise_data/saildrone/noaa_arctic/PMEL_2016/128/pmel_2016_sd128-ALL-1_min-v1.nc',
#                         'F:/data/cruise_data/saildrone/2019_arctic/daily_files/arctic_2019_sd1033-ALL-1_min-v1.nc',
#                         'F:/data/cruise_data/saildrone/2019_arctic/daily_files/arctic_2019_sd1034-ALL-1_min-v1.nc',
#                         'F:/data/cruise_data/saildrone/2019_arctic/daily_files/arctic_2019_sd1035-ALL-1_min-v1.nc',
#                         'F:/data/cruise_data/saildrone/2019_arctic/daily_files/arctic_2019_sd1036-ALL-1_min-v1.nc',
#                         'F:/data/cruise_data/saildrone/2019_arctic/daily_files/arctic_2019_sd1037-ALL-1_min-v1.nc',
#                        'F:/data/cruise_data/saildrone/antarctic/saildrone-gen_5-antarctica_circumnavigation_2019-sd1020-20190119T040000-20190803T043000-1440_minutes-v1.1564857794963.nc']
    filename_usv_list = ['C:/Users/gentemann/Google Drive/private/tem_saildrone/pmel_2015_sd126-ALL-1_min-v1.nc',
                         'C:/Users/gentemann/Google Drive/private/tem_saildrone/pmel_2015_sd128-ALL-1_min-v1.nc',
                         'C:/Users/gentemann/Google Drive/private/tem_saildrone/pmel_2016_sd126-ALL-1_min-v1.nc',
                         'C:/Users/gentemann/Google Drive/private/tem_saildrone/pmel_2016_sd128-ALL-1_min-v1.nc',
                         'C:/Users/gentemann/Google Drive/private/tem_saildrone/arctic_2019_sd1033-ALL-1_min-v1.nc',
                         'C:/Users/gentemann/Google Drive/private/tem_saildrone/arctic_2019_sd1034-ALL-1_min-v1.nc',
                         'C:/Users/gentemann/Google Drive/private/tem_saildrone/arctic_2019_sd1035-ALL-1_min-v1.nc',
                         'C:/Users/gentemann/Google Drive/private/tem_saildrone/arctic_2019_sd1036-ALL-1_min-v1.nc',
                         'C:/Users/gentemann/Google Drive/private/tem_saildrone/arctic_2019_sd1037-ALL-1_min-v1.nc',
                         'C:/Users/gentemann/Google Drive/private/tem_saildrone/saildrone-gen_5-antarctica_circumnavigation_2019-sd1020-20190119T040000-20190803T043000-1440_minutes-v1.1564857794963.nc']
    name_usv_list = ['arctic2015_126',
                     'arctic2015_128',
                     'arctic2016_126',
                     'arctic2016_128',
                     'arctic2019_1033',
                     'arctic2019_1034',
                     'arctic2019_1035',
                     'arctic2019_1036',
                     'arctic2019_1037',
                    'antarctic2019']

    filename_usv = filename_usv_list[iusv]
#    if iusv==3:
#    elif iusv<3:
#    elif (iusv>3) & (iusv<8):
    ds_usv = xr.open_dataset(filename_usv)
    ds_usv.close()
#NEED TO FIND OUT IF wind_speed is to/from wind_direction ?
    if (iusv==0 or iusv==1):  #1033
        ds_usv = ds_usv.rename({'temp_air_mean':'TEMP_AIR_MEAN','rh_mean':'RH_MEAN','baro_pres_mean':'BARO_PRES_MEAN',
                                'sal_mean':'SAL_MEAN','temp_ctd_mean':'TEMP_CTD_MEAN','temp_o2_mean':'TEMP_O2_MEAN',
                                'chlor_mean':'CHLOR_MEAN','gust_wnd_mean':'GUST_WND_MEAN'})
        tem_att=ds_usv.wind_speed_mean.attrs
        ds_usv['wind_speed_mean']=ds_usv.wind_speed_mean*.51444
        ds_usv.wind_speed_mean.attrs=tem_att
        ds_usv.wind_speed_mean.attrs['units']='m s-1'
        uwnd = ds_usv.wind_speed_mean*np.cos(np.deg2rad(ds_usv.wind_direction_mean))
        vwnd = ds_usv.wind_speed_mean*np.sin(np.deg2rad(ds_usv.wind_direction_mean))
        ds_usv['UWND_MEAN']=uwnd
        ds_usv.UWND_MEAN.attrs={'standard_name':'eastward_wind','long_name':'Eastward wind speed','units':'m s-1','installed_height':'5.2'}
        ds_usv['VWND_MEAN']=vwnd
        ds_usv.VWND_MEAN.attrs={'standard_name':'northward_wind','long_name':'Northward wind speed','units':'m s-1','installed_height':'5.2'}
        ilen = ds_usv.time.shape[0]
        ds_usv['WWND_MEAN']=xr.DataArray(np.ones(ilen)*np.nan,coords={'time':ds_usv.time},dims=('time'))
        ds_usv.WWND_MEAN.attrs={'standard_name':'upward_wind_velocity','long_name':'upward wind speed','units':'m s-1','installed_height':'5.2'}
    if (iusv==2 or iusv==3):  #1033
        ds_usv = ds_usv.rename({'temp_air_mean':'TEMP_AIR_MEAN','rh_mean':'RH_MEAN','baro_pres_mean':'BARO_PRES_MEAN',
                                'sal_mean':'SAL_MEAN','temp_ctd_mean':'TEMP_CTD_MEAN','temp_o2_mean':'TEMP_O2_MEAN',
                                'chlor_mean':'CHLOR_MEAN','gust_wnd_mean':'GUST_WND_MEAN'})
        tem_att=ds_usv.wind_speed.attrs
        ds_usv['wind_speed']=ds_usv.wind_speed*.51444
        ds_usv.wind_speed.attrs=tem_att
        ds_usv.wind_speed.attrs['units']='m s-1'
        uwnd = ds_usv.wind_speed*np.cos(np.deg2rad(ds_usv.wind_direction))
        vwnd = ds_usv.wind_speed*np.sin(np.deg2rad(ds_usv.wind_direction))
        ds_usv['UWND_MEAN']=uwnd
        ds_usv.UWND_MEAN.attrs={'standard_name':'eastward_wind','long_name':'Eastward wind speed','units':'m s-1','installed_height':'5.2'}
        ds_usv['VWND_MEAN']=vwnd
        ds_usv.VWND_MEAN.attrs={'standard_name':'northward_wind','long_name':'Northward wind speed','units':'m s-1','installed_height':'5.2'}
        ilen = ds_usv.time.shape[0]
        ds_usv['WWND_MEAN']=xr.DataArray(np.ones(ilen)*np.nan,coords={'time':ds_usv.time},dims=('time'))
        ds_usv.WWND_MEAN.attrs={'standard_name':'upward_wind_velocity','long_name':'upward wind speed','units':'m s-1','installed_height':'5.2'}
    if iusv==4:  #1033
        ds_usv = ds_usv.rename({'TEMP_CTD_RBR_MEAN':'TEMP_CTD_MEAN','TEMP_O2_RBR_MEAN':'TEMP_O2_MEAN','SAL_RBR_MEAN':'SAL_MEAN','CHLOR_WETLABS_MEAN':'CHLOR_MEAN'})
    if iusv==5:  #1034
        ds_usv = ds_usv.rename({'TEMP_CTD_RBR_MEAN':'TEMP_CTD_MEAN','TEMP_O2_RBR_MEAN':'TEMP_O2_MEAN','SAL_RBR_MEAN':'SAL_MEAN','CHLOR_WETLABS_MEAN':'CHLOR_MEAN'})
    if iusv==6:  #1035
        ds_usv = ds_usv.rename({'TEMP_CTD_RBR_MEAN':'TEMP_CTD_MEAN','TEMP_O2_RBR_MEAN':'TEMP_O2_MEAN','SAL_RBR_MEAN':'SAL_MEAN','CHLOR_WETLABS_MEAN':'CHLOR_MEAN'})
    if iusv==7:  #1036
        ds_usv = ds_usv.isel(time=slice(100,-1))                                                                   #        ds_usv = ds_usv.rename({'TEMP_CTD_RBR_MEAN':'TEMP_CTD_MEAN','TEMP_O2_RBR_MEAN':'TEMP_O2_MEAN','SAL_RBR_MEAN':'SAL_MEAN','CHLOR_WETLABS_MEAN':'CHLOR_MEAN'})
        ds_usv = ds_usv.rename({'TEMP_CTD_RBR_MEAN':'TEMP_CTD_MEAN','TEMP_O2_RBR_MEAN':'TEMP_O2_MEAN','SAL_RBR_MEAN':'SAL_MEAN','CHLOR_WETLABS_MEAN':'CHLOR_MEAN'})
    if iusv==8:  #1037
        ds_usv = ds_usv.rename({'TEMP_CTD_RBR_MEAN':'TEMP_CTD_MEAN','TEMP_O2_RBR_MEAN':'TEMP_O2_MEAN'})
    if iusv==9:  #1037
        ds_usv = ds_usv.isel(trajectory=0).swap_dims({'obs':'time'}).rename({'latitude':'lat','longitude':'lon','TEMP_O2_RBR_MEAN':'TEMP_O2_MEAN'})#TEMP_CTD_RBR_MEAN':'TEMP_
    if (iusv==9 or iusv<=3):
        ilen = ds_usv.time.shape[0]
        ds_usv['WIND_HEIGHT_MEAN']=xr.DataArray(np.ones(ilen)*np.nan,coords={'time':ds_usv.time},dims=('time'))
        ds_usv.WIND_HEIGHT_MEAN.attrs={'long_name':'Wind measurement height','units':'m','installed_height':'5.2'}
        ds_usv['WAVE_DOMINANT_PERIOD']=xr.DataArray(np.ones(ilen)*np.nan,coords={'time':ds_usv.time},dims=('time'))
        ds_usv.WAVE_DOMINANT_PERIOD.attrs={'standard_name':'sea_surface_wave_period_at_variance_spectral_density_maximum','long_name':'Dominant wave period','units':'s','installed_height':'0.34'}
        ds_usv['WAVE_SIGNIFICANT_HEIGHT']=xr.DataArray(np.ones(ilen)*np.nan,coords={'time':ds_usv.time},dims=('time'))
        ds_usv.WAVE_SIGNIFICANT_HEIGHT.attrs={'standard_name':'sea_surface_wave_significant_height','long_name':'Significant wave height','units':'m','installed_height':'0.34'}

    #add room to write collocated data information
    ilen = ds_usv.time.shape[0]
    ds_usv['deltaT']=xr.DataArray(np.ones(ilen)*999999,coords={'time':ds_usv.time},dims=('time'))
    ds_usv['amsr2_name']=xr.DataArray(np.empty(ilen,dtype=str),coords={'time':ds_usv.time},dims=('time'))
    ds_usv['amsr2_dist']=xr.DataArray(np.ones(ilen)*999999,coords={'time':ds_usv.time},dims=('time'))
    ds_usv['amsr2_ydim']=xr.DataArray(np.ones(ilen)*999999,coords={'time':ds_usv.time},dims=('time'))
    ds_usv['amsr2_xdim']=xr.DataArray(np.ones(ilen)*999999,coords={'time':ds_usv.time},dims=('time'))

    return ds_usv,name_usv_list[iusv]

In [3]:
ds_usv,name=read_usv(9)
ds_usv
#print(ds_usv.trajectory)
#ds_usv
#ds_usv.WAVE_SIGNIFICANT_HEIGHT

#(ds_usv.TEMP_SBE37_MEAN-ds_usv.TEMP_O2_RBR_MEAN).plot()
#(ds_usv.TEMP_CTD_RBR_MEAN-ds_usv.TEMP_O2_RBR_MEAN).plot()
#(ds_usv.TEMP_SBE37_MEAN-ds_usv.TEMP_O2_RBR_MEAN).plot()
#ds_usv.TEMP_O2_RBR_MEAN[100:].plot()
#(ds_usv.TEMP_CTD_RBR_MEAN-ds_usv.TEMP_O2_RBR_MEAN).plot()
#(ds_usv.TEMP_O2_AANDERAA_MEAN-ds_usv.TEMP_O2_RBR_MEAN).plot()
#(ds_usv.TEMP_CTD_RBR_MEAN-ds_usv.TEMP_SBE37_MEAN).plot()
#(ds_usv.SAL_RBR_MEAN-ds_usv.SAL_SBE37_MEAN).plot()
#(ds_usv.SAL_MEAN-ds_usv.SAL_SBE37_MEAN).plot()
#(ds_usv.SAL_RBR_MEAN).plot()
#(ds_usv.SAL_SBE37_MEAN).plot()
#(ds_usv.CHLOR_MEAN-ds_usv.CHLOR_RBR_MEAN).plot()
#ds_usv.CHLOR_MEAN.plot()
#ds_usv.CHLOR_RBR_MEAN.plot()
#(ds_usv.CHLOR_RBR_MEAN-ds_usv.CHLOR_WETLABS_MEAN).plot()
#(ds_usv.CHLOR_RBR_MEAN).plot()
#(ds_usv.CHLOR_WETLABS_MEAN).plot(2
#subset=ds_usv.sel(time=slice('2019-06-15T20','2019-06-15T22'))
#(subset.TEMP_SBE37_MEAN-subset.TEMP_O2_RBR_MEAN).plot()
#(subset.TEMP_CTD_RBR_MEAN-subset.TEMP_O2_RBR_MEAN).plot()
#(subset.TEMP_SBE37_MEAN-subset.TEMP_CTD_RBR_MEAN).plot()
#(subset.TEMP_O2_RBR_MEAN-subset.TEMP_CTD_RBR_MEAN).plot()
#(subset.TEMP_O2_RBR_MEAN-subset.TEMP_SBE37_MEAN).plot()
#(subset.TEMP_CTD_RBR_MEAN-subset.TEMP_SBE37_MEAN).plot()
#(subset.TEMP_CTD_RBR_MEAN).plot()
#subset.TEMP_O2_RBR_MEAN.plot()
#subset.TEMP_SBE37_MEAN.plot()
#subset=ds_usv.sel(time=slice('2019-07-05T20','2019-07-15T22'))
#(subset.SAL_MEAN-subset.SAL_SBE37_MEAN).plot()
#(subset.SAL_MEAN).plot()
#(subset.SAL_SBE37_MEAN).plot()

<xarray.Dataset>
Dimensions:                        (time: 196)
Coordinates:
    trajectory                     float32 1020.0
  * time                           (time) datetime64[ns] 2019-01-20 ... 2019-08-03
    lat                            (time) float64 ...
    lon                            (time) float64 ...
Data variables:
    SOG                            (time) float64 ...
    COG                            (time) float64 ...
    HDG                            (time) float64 ...
    HDG_WING                       (time) float64 ...
    ROLL                           (time) float64 ...
    PITCH                          (time) float64 ...
    WING_ANGLE                     (time) float64 ...
    BARO_PRES_MEAN                 (time) float64 ...
    BARO_PRES_STDDEV               (time) float64 ...
    TEMP_AIR_MEAN                  (time) float64 ...
    TEMP_AIR_STDDEV                (time) float64 ...
    RH_MEAN                        (time) float64 ...
    RH_STDDEV     

In [12]:
#ds2=xr.open_dataset('c:/Users/gentemann/Desktop/temp/mmd06b_sst_drifter-sst_amsr2-gcw1_2012-190_2012-196.nc')
#ds2['insitu.id']
ds=xr.open_dataset('C:/Users/gentemann/Google Drive/private/tem_saildrone/arctic2015_126AMSR2MMDB_filesave2.nc')
ds.filenames

<xarray.DataArray 'filenames' (dim_0: 206)>
array(['d:/amsr2/L1r/v2/2015/05/12\\GW1AM2_201505121301_075D_L1SGRTBR_2220220.h5.gz',
       'd:/amsr2/L1r/v2/2015/05/12\\GW1AM2_201505122344_171A_L1SGRTBR_2220220.h5.gz',
       'd:/amsr2/L1r/v2/2015/05/13\\GW1AM2_201505131344_082D_L1SGRTBR_2220220.h5.gz',
       ...,
       'd:/amsr2/L1r/v2/2015/07/26\\GW1AM2_201507262327_168A_L1SGRTBR_2220220.h5.gz',
       'd:/amsr2/L1r/v2/2015/07/27\\GW1AM2_201507271327_079D_L1SGRTBR_2220220.h5.gz',
       'd:/amsr2/L1r/v2/2015/07/27\\GW1AM2_201507272231_159A_L1SGRTBR_2220220.h5.gz'],
      dtype=object)
Dimensions without coordinates: dim_0


## explore the in situ data and quickly plot using cartopy
    

In [ ]:
ds_usv,name=read_usv(4)
print(ds_usv.time.min().dt.dayofyear.data,ds_usv.time.max().dt.dayofyear.data)
ds_usv,name=read_usv(6)
print(ds_usv.time.min().dt.dayofyear.data,ds_usv.time.max().dt.dayofyear.data)
#ds = xr.open_dataset('https://podaac-opendap.jpl.nasa.gov:443/opendap/allData/insitu/L2/spurs2/saildrone/SPURS2_Saildrone1006.nc')
#ds


In [ ]:
#ds_usv.time.plot()
#subset = ds_usv.sel(time=slice('2018-01-01','2019-01-01'))
#print(subset.time.dt.dayofyear)
#print(ds_usv.time[0:100].dt.year,ds_usv.time[0:100].dt.dayofyear)
#ds_usv.where(ds_usv['lon']>-180)

In [ ]:
for iusv in range(0,5):
    ds_usv,usvname = read_usv(iusv)
    print(usvname)
    #plot cruise SSS with coastlines
    if iusv<9:
        ax = plt.axes(projection=ccrs.PlateCarree())
    if iusv==9:
        ax = plt.axes(projection=ccrs.SouthPolarStereo())
    #ds_usv = ds_usv.where(np.isfinite(ds_usv.lon))
    cs1 = ax.scatter(ds_usv.lon, ds_usv.lat, s=3.0, c=ds_usv.TEMP_CTD_MEAN, edgecolor='none', cmap='jet',vmin=0,vmax=12)
    ax.coastlines(resolution='50m')
    x1,x2,y1,y2 = ds_usv.lon.min().data-2,ds_usv.lon.max().data+2,ds_usv.lat.min().data-2,ds_usv.lat.max().data+2
    ax.set_xlim(x1,x2)
    ax.set_ylim(y1,y2)
    ax.set_xticks(np.arange(x1,x2,4))
    ax.set_yticks(np.arange(y1,y2,5))
    cax = plt.colorbar(cs1)
    cax.set_label('SST (K)')
    fig_fname = 'C:/Users/gentemann/Google Drive/f_drive/docs/projects/misst-arctic/mmdb/'+usvname+'_location.png'
    plt.savefig(fig_fname, transparent=False, format='png')
    plt.clf()

In [4]:
for iusv in range(9,10):
    ds_usv,usvname = read_usv(iusv)
    print(usvname)
    ax = plt.axes(projection=ccrs.SouthPolarStereo())
    cs1 = ax.scatter(ds_usv.lon, ds_usv.lat, transform=ccrs.PlateCarree(),s=10.0, c=ds_usv.TEMP_CTD_MEAN, edgecolor='none', cmap='jet',vmin=0,vmax=12)
    ax.set_extent([-180, 180, -90, -45], crs=ccrs.PlateCarree())
    ax.coastlines(resolution='50m')
    cax = plt.colorbar(cs1)
    cax.set_label('SST (K)')
    fig_fname = 'C:/Users/gentemann/Google Drive/f_drive/docs/projects/misst-arctic/mmdb/'+usvname+'_location.png'
    plt.savefig(fig_fname, transparent=False, format='png')
    plt.clf()


antarctic2019


<Figure size 432x288 with 0 Axes>

In [13]:
for iusv in range(9,10):
    ds_usv,usvname = read_usv(iusv)
    print(usvname)
    ax = plt.axes(projection=ccrs.SouthPolarStereo())
    cs1 = ax.scatter(ds_usv.lon, ds_usv.lat, transform=ccrs.PlateCarree(),s=10.0, c=ds_usv.SAL_MEAN, edgecolor='none', cmap='jet',vmin=33.6,vmax=34.4)
    ax.set_extent([-180, 180, -90, -45], crs=ccrs.PlateCarree())
    ax.coastlines(resolution='50m')
    cax = plt.colorbar(cs1)
    cax.set_label('SSS (psu)')
    fig_fname = 'C:/Users/gentemann/Google Drive/f_drive/docs/projects/misst-arctic/mmdb/'+usvname+'_sal_location.png'
    plt.savefig(fig_fname, transparent=False, format='png')
    plt.clf()


antarctic2019


<Figure size 432x288 with 0 Axes>

## An example showing how the using matplotlib maps orbital data quickly and easily



In [ ]:
ds_usv,usvname = read_usv(7)
adir_in = 'd:/'
#adir_in = 'C:/Users/gentemann/Google Drive/public/temp/'
file=adir_in+'amsr2_update/ftp.gportal.jaxa.jp/standard/GCOM-W/GCOM-W.AMSR2/L1R/2/2019/05/GW1AM2_201905301307_076D_L1SGRTBR_2220220.h5'
ds = xr.open_dataset(file)
ds.close()
xlat=ds['Latitude of Observation Point for 89A'][:,::2]
xlon=ds['Longitude of Observation Point for 89A'][:,::2]
x = xlon.data
y = xlat.data
z = ds['Brightness Temperature (res06,6.9GHz,H)'].data*.01
ax = plt.axes(projection=ccrs.PlateCarree())
cs1 = ax.scatter(x, y, s=1.0, c=z, edgecolor='none', cmap='jet')
minlon,maxlon,minlat,maxlat = ds_usv.lon.min().data,ds_usv.lon.max().data,ds_usv.lat.min().data,ds_usv.lat.max().data
ax.plot([minlon,maxlon,maxlon,minlon,minlon],[minlat,minlat,maxlat,maxlat,minlat])
ax.plot(ds_usv.lon,ds_usv.lat,'w')
subset=ds_usv.sel(time=slice('2019-05-29','2019-05-31'))
ax.plot(subset.lon,subset.lat,'m')
#ax.plot(ds.cellon[jj,ii],ds.cellat[jj,ii,0],'b*')
#ax.plot(ds_usv.lon[1000],ds_usv.lat[1000],'ro')
ax.coastlines(resolution='50m')
ax.set_xlim(-175,-145)
ax.set_ylim(50,78)
cax = plt.colorbar(cs1)
cax.set_label('TB 6.9H (K)')
fig_fname = 'C:/Users/gentemann/Google Drive/f_drive/docs/projects/misst-arctic/mmdb/example_collocation.png'
plt.savefig(fig_fname, transparent=False, format='png')


## But you can't search off of a scatter plot, so let's use pyresample to quickly project the orbit

In [ ]:
area_def = load_area('areas.cfg', 'pc_world')
rlon=np.arange(-180,180,.1)
rlat=np.arange(90,-90,-.1)
#adir_in = 'C:/Users/gentemann/Google Drive/public/temp/'
adir_in = 'd:/'
file=adir_in+'amsr2_update/ftp.gportal.jaxa.jp/standard/GCOM-W/GCOM-W.AMSR2/L1R/2/2019/05/GW1AM2_201905301307_076D_L1SGRTBR_2220220.h5'
ds = xr.open_dataset(file)
ds.close()
xlat=ds['Latitude of Observation Point for 89A'][:,::2]
xlon=ds['Longitude of Observation Point for 89A'][:,::2]
x = xlon.data
y = xlat.data
z = ds['Brightness Temperature (res06,6.9GHz,H)'].data*.01
#x = (np.mod(ds.cellon+180,360)-180).data 
lons,lats,data = x,y,z 
swath_def = SwathDefinition(lons, lats)
result = resample_nearest(swath_def, data, area_def, radius_of_influence=20000, fill_value=None)
save_quicklook('C:/Users/gentemann/Google Drive/f_drive/docs/projects/misst-arctic/mmdb/6hplot.png', area_def, result, num_meridians=0, num_parallels=0, label='Salinity (psu)')
#now plot on basemap
bmap = area_def2basemap(area_def)
bmng = bmap.bluemarble()
col = bmap.imshow(result, origin='upper',vmin=100,vmax=250)

## First let's figure out what orbital files actually have data in our area of interest.  To do this, use the pyresample software

- read in the in situ data
- calculate the in situ min/max dates to know what files to check

Now we have our time of interest

- loop through the satellite data
- calculate the in situ min/max lat/lon on the same day to define a small box of interest
- use pyresample to map the data onto a predefined 0.1 deg resolution spatial grid
- subset the gridded map to the area of interest
- see if there is any valid data in that area
- if there is any valid data, save the filename into a list



In [ ]:
#intialize grid
for iusv in range(0,1):
    area_def = load_area('areas.cfg', 'pc_world')
    rlon=np.arange(-180,180,.1)
    rlat=np.arange(90,-90,-.1)

    ds_usv,name_usv = read_usv(iusv)

#    adir = 'C:/Users\gentemann/Google Drive/public/temp/'
    adir = 'd:/'
    if ds_usv.time.min().dt.year.data<2018:
        sat_directory=adir+'amsr2/L1r/v2/'
    else:
        sat_directory=adir+'amsr2_update/ftp.gportal.jaxa.jp/standard/GCOM-W/GCOM-W.AMSR2/L1R/2/'
    fileout = 'C:/Users\gentemann/Google Drive/private/tem_saildrone/'+name_usv+'AMSR2MMDB_filesave2.nc'
    file_end = '*.h5.gz'
    
#    if path.exists(fileout):
#        continue
    #init filelist
    file_save=[]

    #search usv data
    minday,maxday = ds_usv.time[0],ds_usv.time[-1]
    usv_day = minday
    print(minday.data,maxday.data)
    while usv_day<=maxday:
        
#        check_day = np.datetime64(str(usv_day.dt.year.data)+'-'+str(usv_day.dt.month.data).zfill(2)+'-'+str(usv_day.dt.day.data).zfill(2))
#        usv_day1 = usv_day + np.timedelta64(1,'D')
#        check_day1 = np.datetime64(str(usv_day1.dt.year.data)+'-'+str(usv_day1.dt.month.data).zfill(2)+'-'+str(usv_day1.dt.day.data).zfill(2))
#        ds_day = ds_usv.sel(time=slice(check_day,check_day1))

#while looping through USV data, look at data +-1 day 
        ds_day = ds_usv.sel(time=slice(usv_day-np.timedelta64(1,'D'),usv_day+np.timedelta64(1,'D')))
        ilen = ds_day.time.size
        if ilen<1:   #don't run on days without any data
            continue
        minlon,maxlon,minlat,maxlat = ds_day.lon.min().data,ds_day.lon.max().data,ds_day.lat.min().data,ds_day.lat.max().data
        #caluclate filelist
        syr=str(usv_day.dt.year.data)
        smon=str(usv_day.dt.month.data).zfill(2)
        sdy=str(usv_day.dt.day.data).zfill(2)
        #the more recent data is in daily directories, so easy to search
        #the older data, pre 2018 is in monthly directories so only search files for day
        if usv_day.dt.year.data<2018:
            adir_list=sat_directory+syr+'/'+smon+'/'+sdy+'/'+file_end
            filelist = glob(adir_list)
        else:
            adir_list=sat_directory+syr+'/'+smon+'/'+'/GW1AM2_'+syr+smon+sdy+file_end
            filelist = glob(adir_list)  
        #print(sat_directory+syr+'/'+smon+'/'+'/GW1AM2_'+syr+smon+sdy+file_end)
        print(usv_day.data,'numfiles:',len(filelist))
        print(adir_list)
        
        x,y,z = [],[],[]
        for file in filelist:
            file.replace('\\','/')
            with gzip.open(file, 'rb') as f_in:
                with open('c:/temp/tem.h5', 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
            ds=xr.open_dataset('c:/temp/tem.h5')
            ds.close()
#            ds = xr.open_dataset(file)
#            ds.close()
            xlat=ds['Latitude of Observation Point for 89A'][:,::2]
            xlon=ds['Longitude of Observation Point for 89A'][:,::2]
#            ds = ds.rename({'cellon':'lon','cellat':'lat','sss_smap':'smap_sss'})
#            ds['lon']=np.mod(ds.lon+180,360)-180  
#            x = xlon.fillna(-89).data 
#            y = xlat.fillna(-89).data 
            x = xlon.data
            y = xlat.data
            z = ds['Brightness Temperature (res06,6.9GHz,H)'].data*.01 
            lons,lats,data = x,y,z 
            swath_def = SwathDefinition(lons, lats)
            result1 = resample_nearest(swath_def, data, area_def, radius_of_influence=20000, fill_value=None)
            da = xr.DataArray(result1,name='tb6h',coords={'lat':rlat,'lon':rlon},dims=('lat','lon'))
            subset = da.sel(lat = slice(maxlat,minlat),lon=slice(minlon,maxlon))
            num_obs = np.isfinite(subset).sum()
            if num_obs>0:
                file_save = np.append(file_save,file)
        usv_day += np.timedelta64(1,'D')
        df = xr.DataArray(file_save,name='filenames')
        df.to_netcdf(fileout)

In [1]:
ds_usv,usvname = read_usv(7)
ds_usv

NameError: name 'read_usv' is not defined

## Now, loop through only the files that we know have some data in the region of interest.  Use the fast search kdtree which is part of pyresample software, but I think maybe comes originally from sci-kit-learn.

- read in the in situ data
- read in a single orbit of satellite data
- kdtree can't handle it when lat/lon are set to nan.  I frankly have no idea why there is orbital data for both the JPL and RSS products that have nan for the geolocation.  That isn't normal.  But, okay, let's deal with it.  
- stack the dataset scanline and cell positions into a new variable 'z'
- drop all variables from the dataset when the longitude is nan
- set up the tree
- loop through the orbital data
- only save a match if it is less than 0.25 deg distance AND time is less than any previous match
- save the satellite indices & some basic data onto the USV grid


In [ ]:
num_usv=0
for isat in range(2):`
    ds_usv,usv_name = read_usv(num_usv)
    filelist = 'F:/data/cruise_data/saildrone/sat_collocations/'+usv_name+'AMSR2MMDB_filesave2.nc'
    fileout = 'F:/data/cruise_data/saildrone/sat_collocations/'+usv_name+'AMSR2MMDB_usv2.nc'   
    df = xr.open_dataset(filelist)
    for file2 in df.filenames.data:
        file = file2
        file.replace('\\','/')
        ds = xr.open_dataset(file)
        ds.close()  
        if isat==0:  #change RSS data to conform with JPL definitions
            ds = ds.rename({'iqc_flag':'quality_flag','cellon':'lon','cellat':'lat','sss_smap':'smap_sss','ydim_grid':'phony_dim_0','xdim_grid':'phony_dim_1'})
            ds['lon']=np.mod(ds.lon+180,360)-180  
        if isat==1:  #change RSS data to conform with JPL definitions
            ds = ds.rename({'row_time':'time'})
        #stack xarray dataset then drop lon == nan
        ds2 = ds.stack(z=('phony_dim_0', 'phony_dim_1')).reset_index('z')
        #drop nan
        ds_drop = ds2.where(np.isfinite(ds2.lon),drop=True)
        lats = ds_drop.lat.data
        lons = ds_drop.lon.data
        inputdata = list(zip(lons.ravel(), lats.ravel()))
        tree = spatial.KDTree(inputdata)
        orbit_time = ds.time.max().data-np.timedelta64(1,'D')
        orbit_time2 = ds.time.max().data+np.timedelta64(1,'D')    
        usv_subset = ds_usv.sel(time=slice(orbit_time,orbit_time2))
        ilen = ds_usv.time.size
        for iusv in range(ilen):
            if (ds_usv.time[iusv]<orbit_time) or (ds_usv.time[iusv]>orbit_time2):
                continue
            pts = np.array([ds_usv.lon[iusv], ds_usv.lat[iusv]])
    #        pts = np.array([ds_usv.lon[iusv]+360, ds_usv.lat[iusv]])
            tree.query(pts,k=1)
            i = tree.query(pts)[1]
            rdist = tree.query(pts)[0]
            #don't use matchups more than 25 km away
            if rdist>.25:
                continue
            #use .where to find the original indices of the matched data point
            #find by matching sss and lat, just randomly chosen variables, you could use any
            result = np.where((ds.smap_sss == ds_drop.smap_sss[i].data) & (ds.lat == ds_drop.lat[i].data))
            listOfCoordinates = list(zip(result[0], result[1]))
            if len(listOfCoordinates)==0:
                continue
            ii, jj = listOfCoordinates[0][0],listOfCoordinates[0][1]
            if isat==0:
                deltaTa = ((ds_usv.time[iusv]-ds.time[ii,jj]).data)/ np.timedelta64(1,'m')
            if isat==1:
                deltaTa = ((ds_usv.time[iusv]-ds.time[ii]).data)/ np.timedelta64(1,'m')
            if np.abs(deltaTa)<np.abs(ds_usv.deltaT[iusv].data):
                ds_usv.deltaT[iusv]=deltaTa
                ds_usv.smap_SSS[iusv]=ds.smap_sss[ii,jj]
                ds_usv.smap_iqc_flag[iusv]=ds.quality_flag[ii,jj]
                ds_usv.smap_name[iusv]=file2
                ds_usv.smap_dist[iusv]=rdist
                ds_usv.smap_ydim[iusv]=ii
                ds_usv.smap_xdim[iusv]=jj
    ds_usv.to_netcdf(fileout)


# A larger STD that isn't reflective of uncertainty in the observation
The collocation above will result in multiple USV data points matched with a single satellite
observation.    The USV is sampling every 1 min and approximately few meters, while the satellite
is an average over a footprint that is interpolated onto a daily mean map.  While calculating the mean would results in a valid mean, the STD would be higher and consist of a component that reflects the uncertainty of the USV and the satellite and a component that reflects the natural variability in the region that is sampled by the USV

Below we use the 'nearest' collocation results to identify when multiple USV data are collcated to
a single satellite observation.
This code goes through the data and creates averages of the USV data that match the single CCMP collocated value.


In [ ]:
for isat in range(2):
    if isat==0:
        file = 'F:/data/cruise_data/saildrone/sat_collocations/rss40km_usv2.nc'
        fileout = 'F:/data/cruise_data/saildrone/sat_collocations/rss40km_usv2_norepeats.nc'
    if isat==1:
        file = 'F:/data/cruise_data/saildrone/sat_collocations/jplv42_usv2.nc'   
        fileout = 'F:/data/cruise_data/saildrone/sat_collocations/jplv42_usv2_norepeats.nc'   
    ds_usv=xr.open_dataset(file)
    ds_usv.close()
    ds_usv = ds_usv.where(ds_usv.smap_SSS<10000,np.nan)
    ilen,index = ds_usv.dims['time'],0
    ds_tem = ds_usv.copy(deep=True)
    duu, duu2, duv1, duv2, dlat, dlon, dut = [],[],[],[],[],[],np.empty((),dtype='datetime64')
    index=0
    while index <= ilen-2:
        index += 1
        if np.isnan(ds_usv.smap_SSS[index]):
            continue
        if np.isnan(ds_usv.smap_xdim[index]):
            continue
        result = np.where((ds_usv.smap_xdim == ds_tem.smap_xdim[index].data) & (ds_usv.smap_ydim == ds_tem.smap_ydim[index].data))       
        duu=np.append(duu,ds_usv.smap_SSS[result[0][0]].data)
        duu2=np.append(duu2,ds_usv.smap_iqc_flag[result[0][0]].data)
        duv1=np.append(duv1,ds_usv.SAL_MEAN[result].mean().data)
        dlat=np.append(dlat,ds_usv.lat[result].mean().data)
        dlon=np.append(dlon,ds_usv.lon[result].mean().data)
        dut=np.append(dut,ds_usv.time[result].mean().data)
        ds_usv.smap_SSS[result]=np.nan
    dut2 = dut[1:]  #remove first data point which is a repeat from what array defined       
    ds_new=xr.Dataset(data_vars={'smap_SSS': ('time',duu),'smap_iqc_flag': ('time',duu2),
                                 'SAL_MEAN':('time',duv1),
                                 'lon': ('time',dlon),
                                 'lat': ('time',dlat)},
                      coords={'time':dut2})
    ds_new.to_netcdf(fileout)



# Now look at bias/std & plot 

In [ ]:
files =['F:/data/cruise_data/saildrone/sat_collocations/rss40km_usv2_norepeats.nc',
        'F:/data/cruise_data/saildrone/sat_collocations/jplv42_usv2_norepeats.nc',
        'F:/data/cruise_data/saildrone/sat_collocations/rss40km_usv2.nc',
        'F:/data/cruise_data/saildrone/sat_collocations/jplv42_usv2.nc']
flabel=['RSS','JPL','RSS','JPL']
for isat in range(4):
    file = files[isat]
    ds_usv=xr.open_dataset(file)
    ds_usv = ds_usv.where(ds_usv.smap_SSS<10000,np.nan)
    ds_usv = ds_usv.where(ds_usv.smap_iqc_flag<9400,np.nan)
    if isat==1 or isat==3:
        ds_usv = ds_usv.where(ds_usv.smap_iqc_flag<1,np.nan)
    tem=ds_usv.smap_SSS-ds_usv.SAL_MEAN
    tem = tem.where(np.isfinite(tem),drop=True)
    print(flabel[isat],tem.mean().data,tem.std().data,tem.size)


# just playing around & plotting stuff

In [ ]:
print(tem[20000].data,ds_usv.smap_iqc_flag[20000].data)

In [ ]:
plt.plot(ds_usv.smap_SSS,'.')
plt.plot(ds_usv.smap_iqc_flag,'.')


In [ ]:
plt.plot(ds_usv.smap_iqc_flag,ds_usv.smap_SSS-ds_usv.SAL_MEAN,'.')


In [ ]:
tem=ds_usv.smap_SSS-ds_usv.SAL_MEAN
plt.scatter(ds_usv.lon,ds_usv.lat,c=tem,vmin=-1,vmax=1)

In [ ]:
#plot cruise SSS with coastlines
ax = plt.subplot(221,projection=ccrs.PlateCarree())
cs1 = ax.scatter(ds_usv.lon, ds_usv.lat, s=3.0, c=ds_usv.SAL_MEAN, edgecolor='none', cmap='jet',vmin=33,vmax=34.35)
ax.coastlines()
x1,x2,y1,y2 = -128,-112,25,40
ax.set_xlim(x1,x2)
ax.set_ylim(y1,y2)
ax.set_xticks(np.arange(x1,x2,4))
ax.set_yticks(np.arange(y1,y2,5))

ax = plt.subplot(222,projection=ccrs.PlateCarree())
cs1 = ax.scatter(ds_usv.lon, ds_usv.lat, s=3.0, c=ds_usv.smap_SSS, edgecolor='none', cmap='jet',vmin=33,vmax=34.35)
ax.coastlines()
x1,x2,y1,y2 = -128,-112,25,40
ax.set_xlim(x1,x2)
ax.set_ylim(y1,y2)
ax.set_xticks(np.arange(x1,x2,4))
ax.set_yticks(np.arange(y1,y2,5))

cax = plt.colorbar(cs1)
cax.set_label('Salinity (psu)')
fig_fname = 'C:/Users/gentemann/Google Drive/f_drive/docs/projects/SSS/figures/baja_location2.png'
plt.savefig(fig_fname, transparent=False, format='png')


In [ ]:
fileout = 'F:/data/cruise_data/saildrone/sat_collocations/jplv42_usv.nc'
ds_usv = xr.open_dataset(fileout)
tem = ds_usv.smap_SSS.where(ds_usv.smap_SSS<45,np.NaN)
tem = tem.where(ds_usv.smap_SSS>28,np.NaN)
print((tem-ds_usv.SAL_MEAN).mean().data,(tem-ds_usv.SAL_MEAN).std().data,(tem-ds_usv.SAL_MEAN).size)


In [ ]:
fileout = 'F:/data/cruise_data/saildrone/sat_collocations/rss40km1_usv.nc'
ds_usv = xr.open_dataset(fileout)
tem = ds_usv.smap_SSS.where(ds_usv.smap_SSS<45,np.NaN)
tem = tem.where(ds_usv.smap_SSS>28,np.NaN)
print((tem-ds_usv.SAL_MEAN).mean().data,(tem-ds_usv.SAL_MEAN).std().data,(tem-ds_usv.SAL_MEAN).size)


In [ ]:
plt.plot(tem.time,tem)
plt.plot(ds_usv.time,ds_usv.SAL_MEAN)

# Read in ACCESS data

In [ ]:
filename = 'F:/data/cruise_data/access/rockfish_casts_2011.nc'
ds = xr.open_dataset(filename)
ds = ds.swap_dims({'row':'time'})
ds = ds.rename({'latitude':'lat','longitude':'lon'})
ds_access1 = ds.copy(deep=True)
filename = 'F:/data/cruise_data/access/rockfish_casts_2015.nc'
ds = xr.open_dataset(filename)
ds = ds.swap_dims({'row':'time'})
ds = ds.rename({'latitude':'lat','longitude':'lon'})
ds_access2 = ds.copy(deep=True)
ds_access = xr.concat((ds_access1,ds_access2),dim='time')

In [ ]:
ds_access

In [ ]:
ds_access.time.dt.month

In [ ]:
plt.plot(ds_access.time.dt.year,ds_access.time.dt.dayofyear,'.')
plt.xlabel('Day of Year')
plt.ylabel('Year')
fig_fname = 'C:/Users/gentemann/Google Drive/f_drive/docs/projects/SSS/figures/access_date.png'
plt.savefig(fig_fname, transparent=False, format='png')

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
cs1 = ax.scatter(ds_access.lon, ds_access.lat, s=3.0, c=ds_access.salinity, edgecolor='none', cmap='jet',vmin=33,vmax=34.35)
ax.coastlines()
x1,x2,y1,y2 = -128,-112,30,50
ax.set_xlim(x1,x2)
ax.set_ylim(y1,y2)
ax.set_xticks(np.arange(x1,x2,4))
ax.set_yticks(np.arange(y1,y2,5))
cax = plt.colorbar(cs1)
cax.set_label('Salinity (psu)')
fig_fname = 'C:/Users/gentemann/Google Drive/f_drive/docs/projects/SSS/figures/access_location.png'
plt.savefig(fig_fname, transparent=False, format='png')

# Read in 2018 Saildrone West Coast Survey

In [ ]:
#https://coastwatch.pfeg.noaa.gov/erddap/
#url = 'https://coastwatch.pfeg.noaa.gov/erddap/tabledap/saildrone_west_coast_survey_2018.nc'
#url = 'https://ferret.pmel.noaa.gov/pmel/erddap/tabledap/saildrone_west_coast_survey_2018.nc'
url = 'https://ferret.pmel.noaa.gov/pmel/erddap/tabledap/saildrone_west_coast_survey_2018'
#url = 'F:/data/cruise_data/saildrone/2018_wcoast/saildrone_west_coast_survey_2018_f374_2e74_3de8.nc'
#url = 'https://ferret.pmel.noaa.gov/pmel/erddap/tabledap/saildrone_west_coast_survey_2018.nc'
ds_usv = xr.open_dataset(url)
ds_usv.close()
ds_usv['lat']=ds_usv['s.latitude']
ds_usv['lon']=ds_usv['s.longitude']
ds_usv['time']=ds_usv['s.time']
ds_usv['trajectory']=ds_usv['s.trajectory']
ds_usv['SAL_MEAN']=ds_usv['s.SAL_MEAN']
ds_usv['TEMP_CTD_MEAN']=ds_usv['s.TEMP_CTD_MEAN']
ds_usv = ds_usv.swap_dims({'s':'time'})
#print(ds_usv.time.min().data,ds_usv.time.max().data)
#ds_usv.load()


In [ ]:
#used this code to figure out where the data goes bad
ilen = ds_usv.SAL_MEAN.size
print(ilen)
tem = np.nan(ilen)
for i in range(787066):
    if ds_usv.SAL_MEAN[i]>20:
        tem[i]=ds_usv.SAL_MEAN[i]
tem[i:]=np.nan

In [ ]:
#there is something bad in the data file above 776100 so subset the data to just the good part
ds_usv2 = ds_usv.isel(time=slice(None,776100))
xlon =  ds_usv2.lon.copy(deep=True)
xlat =  ds_usv2.lat.copy(deep=True)
salinity = ds_usv2.SAL_MEAN.copy(deep=True)
ax = plt.axes(projection=ccrs.PlateCarree())
cs1 = ax.scatter(xlon, xlat, s=3.0, c=salinity, edgecolor='none', cmap='jet',vmin=32,vmax=34.35)
ax.coastlines()
x1,x2,y1,y2 = -130,-114,30,52
ax.set_xlim(x1,x2)
ax.set_ylim(y1,y2)
ax.set_xticks(np.arange(x1,x2,4))
ax.set_yticks(np.arange(y1,y2,5))
cax = plt.colorbar(cs1)
cax.set_label('Salinity (psu)')
fig_fname = 'C:/Users/gentemann/Google Drive/f_drive/docs/projects/SSS/figures/wcoast_location2.png'
plt.savefig(fig_fname, transparent=False, format='png')


In [ ]:
ds_usv2.trajectory[0:5]

In [ ]:
url = 'https://ferret.pmel.noaa.gov/pmel/erddap/tabledap/saildrone_west_coast_survey_2018'
#url = 'F:/data/cruise_data/saildrone/2018_wcoast/saildrone_west_coast_survey_2018_f374_2e74_3de8.nc'
#url = 'https://ferret.pmel.noaa.gov/pmel/erddap/tabledap/saildrone_west_coast_survey_2018.nc'
ds_usv = xr.open_dataset(url)
ds_usv.close()
ds_usv

In [ ]:
url = 'https://ferret.pmel.noaa.gov/generic/erddap/tabledap/saildrone_arctic_gts'
ds_usv = xr.open_dataset(url)
ds_usv = xr.open_dataset(url)
ds_usv.close()
ds_usv['lat']=ds_usv['s.latitude']
ds_usv['lon']=ds_usv['s.longitude']
ds_usv['time']=ds_usv['s.time']
ds_usv['trajectory']=ds_usv['s.trajectory']
ds_usv['SAL_MEAN']=ds_usv['s.SAL_MEAN']
ds_usv['TEMP_CTD_MEAN']=ds_usv['s.TEMP_CTD_MEAN']
#ds_usv = ds_usv.swap_dims({'s':'time'})


In [ ]:
ds_usv

In [ ]:
xlon =  ds_usv.lon.copy(deep=True)
xlat =  ds_usv.lat.copy(deep=True)
salinity = ds_usv.SAL_MEAN.copy(deep=True)
ax = plt.axes(projection=ccrs.PlateCarree())
cs1 = ax.scatter(xlon, xlat, s=3.0, c=salinity, edgecolor='none', cmap='jet',vmin=32,vmax=34.35)
ax.coastlines()
x1,x2,y1,y2 = -130,-114,30,52
ax.set_xlim(x1,x2)
ax.set_ylim(y1,y2)
ax.set_xticks(np.arange(x1,x2,4))
ax.set_yticks(np.arange(y1,y2,5))
cax = plt.colorbar(cs1)
cax.set_label('Salinity (psu)')
fig_fname = 'C:/Users/gentemann/Google Drive/f_drive/docs/projects/SSS/figures/arctic_location.png'
plt.savefig(fig_fname, transparent=False, format='png')

In [ ]:
url = 'F:/data/cruise_data/saildrone/2018_wcoast/saildrone_west_coast_survey_2018_f374_2e74_3de8.nc'
ds_usv = xr.open_dataset(url)
ds_usv.close()
xlon =  ds_usv.longitude.copy(deep=True)
xlat =  ds_usv.latitude.copy(deep=True)
salinity =  ds_usv.SAL_MEAN.copy(deep=True)
ax = plt.axes(projection=ccrs.PlateCarree())
cs1 = ax.scatter(xlon, xlat, s=3.0, c=salinity, edgecolor='none', cmap='jet',vmin=32,vmax=34.35)
ax.coastlines()
x1,x2,y1,y2 = -128,-112,25,40
ax.set_xlim(x1,x2)
ax.set_ylim(y1,y2)
ax.set_xticks(np.arange(x1,x2,4))
ax.set_yticks(np.arange(y1,y2,5))
cax = plt.colorbar(cs1)
cax.set_label('Salinity (psu)')
fig_fname = 'C:/Users/gentemann/Google Drive/f_drive/docs/projects/SSS/figures/wcoast_location.png'
plt.savefig(fig_fname, transparent=False, format='png')
